In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

/Users/dlewis/mlenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id="openai-community/gpt2-xl"
torch_dtype = torch.bfloat16

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch_dtype, use_cache=False).to("mps")

In [4]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=50257, bias=False)
)


In [5]:
print(model.get_input_embeddings())

Embedding(50257, 1600)


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

/Users/dlewis/mlenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
input_sequence = tokenizer.encode(["Hello, world", "Every step moves you forward"])
print(input_sequence)

[15496, 11, 995, 6109, 2239, 6100, 345, 2651]


In [8]:
input_sequence_1 = torch.tensor(tokenizer.encode("Every effort moves you"))
print(input_sequence_1)
input_sequence_2 = torch.tensor(tokenizer.encode("Every day holds a"))
print(input_sequence_2)
input_sequence = torch.stack((input_sequence_1, input_sequence_2))
print(input_sequence)

tensor([6109, 3626, 6100,  345])
tensor([6109, 1110, 6622,  257])
tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [9]:
input_sequence = input_sequence.to("mps")

In [10]:
generation_config = GenerationConfig(output_logits=True, return_dict_in_generate=True)
model_output = model.generate(input_sequence, generation_config=generation_config)
print(model_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/Users/dlewis/mlenv/lib/python3.12/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


GenerateDecoderOnlyOutput(sequences=tensor([[6109, 3626, 6100,  345, 5699,  284,  262, 3061,   13,  198,  198,  464,
         3061,  318,  284,  307,  262, 1266,  345,  460],
        [6109, 1110, 6622,  257,  649, 4427,   13,  198,  198,   40, 1101,  407,
         1654,  703,  890,  314, 1183,  307, 1498,  284]], device='mps:0'), scores=None, logits=(tensor([[ 4.0625,  3.5312, -2.8594,  ..., -6.5938, -5.5625,  1.4453],
        [-1.5703,  0.1592, -4.5625,  ..., -7.0938, -4.9688, -0.5859]],
       device='mps:0', dtype=torch.bfloat16), tensor([[ 7.0000,  6.5312, -1.5547,  ..., -7.8750, -6.6562,  3.5000],
        [ 1.3125,  1.2969, -3.5312,  ..., -6.1250, -5.2500, -0.7812]],
       device='mps:0', dtype=torch.bfloat16), tensor([[ 0.1709,  0.9883, -4.0625,  ..., -5.3750, -5.9375,  0.1113],
        [ 6.7188,  5.0625, -2.0156,  ..., -8.3125, -5.0625,  2.8438]],
       device='mps:0', dtype=torch.bfloat16), tensor([[-0.3105, -0.2432, -4.1562,  ..., -4.2188, -3.7188, -0.4004],
        [ 0.3594

In [11]:
len(model_output['logits'])

16

In [12]:
for sequence in model_output['sequences']:
    print(sequence)
    print(tokenizer.decode(sequence))

tensor([6109, 3626, 6100,  345, 5699,  284,  262, 3061,   13,  198,  198,  464,
        3061,  318,  284,  307,  262, 1266,  345,  460], device='mps:0')
Every effort moves you closer to the goal.

The goal is to be the best you can
tensor([6109, 1110, 6622,  257,  649, 4427,   13,  198,  198,   40, 1101,  407,
        1654,  703,  890,  314, 1183,  307, 1498,  284], device='mps:0')
Every day holds a new challenge.

I'm not sure how long I'll be able to


In [13]:
batch_size = 2

for batch in range(batch_size):
    for logits in model_output['logits']:
        probas = torch.nn.functional.softmax(logits[batch], dim=0)
        prediction = torch.argmax(probas)
        print(tokenizer.decode(prediction))

 closer
 to
 the
 goal
.




The
 goal
 is
 to
 be
 the
 best
 you
 can
 new
 challenge
.




I
'm
 not
 sure
 how
 long
 I
'll
 be
 able
 to


In [14]:
probas, indices = torch.topk(torch.nn.functional.softmax(model_output['logits'][0][0], dim=0), 20)
print(probas)
print(indices)
for index in indices:
    print(tokenizer.decode(index))

tensor([0.2832, 0.2070, 0.0593, 0.0593, 0.0525, 0.0493, 0.0317, 0.0281, 0.0170,
        0.0170, 0.0170, 0.0132, 0.0117, 0.0100, 0.0083, 0.0069, 0.0065, 0.0065,
        0.0063, 0.0059], device='mps:0', dtype=torch.bfloat16)
tensor([ 5699,  2651,   287,  3371,   284,  3812,   530,  2252,   319,   422,
         1497,   656,   510,    11,  1863,   257,    13, 18485,  1683,   290],
       device='mps:0')
 closer
 forward
 in
 towards
 to
 toward
 one
 further
 on
 from
 away
 into
 up
,
 along
 a
.
 farther
 ever
 and


In [15]:
model.config

GPT2Config {
  "_name_or_path": "openai-community/gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.0",
  "use_cache": false,
  "vocab_size": 50257
}

In [16]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=50257, bias=False)
)

In [17]:
model.transformer.h[0]

GPT2Block(
  (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2SdpaAttention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [18]:
del(model)
del(model_output)
torch.mps.empty_cache()

In [19]:
print(torch.mps.driver_allocated_memory())
print(torch.mps.current_allocated_memory())

3300786176
3165837568


In [20]:
def remove_layer(model, layer_number):
    del(model.transformer.h[layer_number])
    model.config.n_layer -= 1

In [21]:
def evaluate_layer_removal():
    for i in range(0,48):
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch_dtype, use_cache=False).to("mps")
        remove_layer(model, i)
        model_output = model.generate(input_sequence, generation_config=generation_config)
        for sequence in model_output['sequences']:
            print(i, tokenizer.decode(sequence))
        del(model)
        del(model_output)
        torch.mps.empty_cache()

In [22]:
evaluate_layer_removal()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 Every effort moves youTheTheTheTheTheTheTheTheTheTheTheTheTheTheTheThe
0 Every day holds aTheTheTheTheTheTheTheTheTheThe"The"The"The


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1 Every effort moves you closer to the goal.

The goal is to be able to move your
1 Every day holds a new challenge.

I'm not sure how long I can keep up.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

45 Every effort moves you closer to your goal.

The more you work, the more you learn
45 Every day holds a new challenge.

I'm not sure how many times I've had to


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


46 Every effort moves you closer to the goal.

The goal is to be able to move your
46 Every day holds a new challenge.

I'm not sure how long I'll be able to


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


47 Every effort moves you closer to your goal.

The more you work, the more you learn
47 Every day holds a new challenge.

I'm not sure if I'll ever be able to
